In [8]:
import requests
from dotenv import load_dotenv
import os 
from pprint import pprint
import json
import pandas as pd

#load hidden credentials and values
load_dotenv() 
imdbapi_key = os.getenv("imdbapi_key")
topimdb = pd.read_csv("IMDb_movies_top.csv")

topimdb.head(5)

,imdb_title_id,original_title,year,date_published,genre,duration,country,language,director,writer,...,metascore.1,stand_avgscore,stand_profit,stand_avg_score_profit,imdb_name_id,category,Name,Birthplace,birthyear,deathyear
0,tt0499549,Avatar,2009,1/15/2010,"Action, Adventure, Fantasy",162,USA,"English, Spanish",James Cameron,James Cameron,...,1.669244,1.702613,35.890502,18.796557,nm0000035,composer,James Horner,NaN,1953.0,2015.0
1,tt4154796,Avengers: Endgame,2019,4/24/2019,"Action, Adventure, Drama",181,USA,"English, Japanese, Xhosa, German","Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely",...,1.386947,1.799177,35.740602,18.769889,NaN,NaN,NaN,NaN,NaN,NaN
2,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,12/16/2015,"Action, Adventure, Sci-Fi",138,USA,English,J.J. Abrams,"Lawrence Kasdan, J.J. Abrams",...,1.499866,1.657542,29.860382,15.758962,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0120338,Titanic,1997,1/16/1998,"Drama, Romance",194,"USA, Mexico, Australia, Canada","English, Swedish, Italian, French",James Cameron,James Cameron,...,1.217569,1.476775,28.713778,15.095277,nm0000035,composer,James Horner,NaN,1953.0,2015.0
4,tt4154756,Avengers: Infinity War,2018,4/25/2018,"Action, Adventure, Sci-Fi",149,USA,English,"Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely",...,0.822353,1.516880,26.010766,13.763823,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
baseurl = "https://imdb-api.com/API/"
trailerext = "Trailer/"
top250ext = "Top250Movies/"
filmid = "tt2488496"

trailer_URL = f"{baseurl}{trailerext}{imdbapi_key}/{filmid}"

response = requests.request("GET", trailer_URL)

trailer_df = pd.json_normalize(json.loads(response.text))
trailer_df

,imDbId,title,fullTitle,type,year,videoId,videoTitle,videoDescription,thumbnailUrl,uploadDate,link,linkEmbed,errorMessage
0,tt2488496,Star Wars: Episode VII - The Force Awakens,Star Wars: Episode VII - The Force Awakens (2015),Movie,2015,vi2762323481,Watch With a STARZ Free Trial,Three decades after the defeat of the Galactic...,https://m.media-amazon.com/images/M/MV5BMTkyNT...,09/10/2016 10:19:12,https://www.imdb.com/video/vi2762323481,https://www.imdb.com/video/imdb/vi2762323481/i...,


In [12]:
from youtube_search import YoutubeSearch

titleist = trailer_df.fullTitle.tolist()

t = titleist[0]
results = YoutubeSearch(f"trailer {t}", max_results=10).to_dict()

results

[{'id': 'sGbxmsDFVnE',
  'thumbnails': ['https://i.ytimg.com/vi/sGbxmsDFVnE/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLA7LluahTUE1NQSAs2f1L-l9GGs3A',
   'https://i.ytimg.com/vi/sGbxmsDFVnE/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBma3YEmCr2qg7_Bsc4jTZAOtJC5A'],
  'title': 'Star Wars: The Force Awakens Trailer (Official)',
  'long_desc': 'Watch the official ',
  'channel': 'Star Wars',
  'duration': '2:36',
  'views': '106,710,534 views',
  'url_suffix': '/watch?v=sGbxmsDFVnE'},
 {'id': 'OMOVFvcNfvE',
  'thumbnails': ['https://i.ytimg.com/vi/OMOVFvcNfvE/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAx4totMnhwHk183ie-kxU_X0wWEw',
   'https://i.ytimg.com/vi/OMOVFvcNfvE/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBWplyMGdX32mcPC59fdPtTlbLPrQ'],
  'title': 'Star Wars: The Force Awakens Official Teaser Trailer #1 (2015) - J.J. Abrams Movie HD',
  'long_desc': 'To celebrate the ',

In [19]:
import os
import google.oauth2.credentials as gcred
import google.auth.transport.requests
from google_auth_oauthlib.flow import InstalledAppFlow 
import googleapiclient.discovery
import datetime
import json

def get_goog_oathcreds(token_txt_dir, creds_json_dir):
    if not os.path.exists(token_txt_dir):
        flow = InstalledAppFlow.from_client_secrets_file(creds_json_dir,
                    scopes=["https://www.googleapis.com/auth/youtube.force-ssl"], 
                            redirect_uri=r"http://localhost/")

        oathcreds = flow.run_local_server()

        with open(token_txt_dir, 'w+') as f:
            json.dump({"refresh_token": oathcreds._refresh_token, "expiry": oathcreds.expiry, \
                        "token": oathcreds.token, "id_token": oathcreds.id_token}, f, indent=4, sort_keys=True, default=str)
    else:
        temp = json.loads(open(token_txt_dir, 'r').read())
        mysecret = json.loads(open(creds_json_dir, 'r').read())['installed']
        oathcreds = gcred.Credentials(
            temp['token'],
            refresh_token=temp['refresh_token'],
            id_token=temp['id_token'],
            token_uri=mysecret['token_uri'],
            client_id=mysecret['client_id'],
            client_secret=mysecret['client_secret'],
            scopes=["https://www.googleapis.com/auth/youtube.force-ssl"], #########################################
        )
        tmpexpiry = temp['expiry'].split('.')[0]
        oathcreds.expiry = datetime.datetime.strptime(tmpexpiry,'%Y-%m-%d %H:%M:%S')
        request = google.auth.transport.requests.Request()
        if oathcreds.expired:
            oathcreds.refresh(request)
    return oathcreds

def create_yt_instance(oathcreds):
    api_service_name = "youtube"
    api_version = "v3"
    youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=oathcreds)
    return youtube

#https://python.gotrained.com/youtube-api-extracting-comments/
def get_video_comments(service, **kwargs):
    comments = []
    results = service.commentThreads().list(**kwargs).execute()

    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break

    return comments


oathcreds = get_goog_oathcreds("youtubetoken.json","youtube.json")
youtube = create_yt_instance(oathcreds)

vidid = results[0]["url_suffix"].split("=")[-1]
vidid

RefreshError: ('invalid_scope: Bad Request', '{\n  "error": "invalid_scope",\n  "error_description": "Bad Request"\n}')

In [20]:
comments = get_video_comments(youtube, part='snippet', videoId=vidid, textFormat='plainText')
pprint(comments)

HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=sGbxmsDFVnE&textFormat=plainText&alt=json returned "Request had insufficient authentication scopes.">

In [21]:
from __future__ import unicode_literals
import youtube_dl

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}
ydl = youtube_dl.YoutubeDL(ydl_opts)
ydl.download(['http://www.youtube.com/watch?v=sGbxmsDFVnE'])


[youtube] sGbxmsDFVnE: Downloading webpage
[download] Destination: Star Wars - The Force Awakens Trailer (Official)-sGbxmsDFVnE.webm
[download] 100% of 2.25MiB in 00:00                  
[ffmpeg] Destination: Star Wars - The Force Awakens Trailer (Official)-sGbxmsDFVnE.mp3
Deleting original file Star Wars - The Force Awakens Trailer (Official)-sGbxmsDFVnE.webm (pass -k to keep)


0